In [155]:
import pandas as pd

df = pd.read_excel('limpaBrasil.xlsx')

#### Padronizando valor de CEP sem '-'

In [156]:
# df['CEP'] = df['CEP'].str.replace(r'(\d{5})(\d{3})', r'\1-\2', regex=True) Esse é pra padronizar COM traçinho
df['Cep'] = df['Cep'].str.replace(r'\D', '', regex=True)

#### Apagando valores nulos e diferentes de 8 caracteres da Coluna CEP

In [157]:
df = df.dropna(subset=['Cep'])

In [158]:
df = df[df['Cep'].str.len() == 8]

#### Padronizando como número

In [159]:
# df['CEP_ARRUMADO'] = df['CEP_ARRUMADO'].astype(int) # Não fazer porque retira o zero na frente do CEP, dificultando achar as informações referentes a ele

#### Verificando em qual zona de SP está

In [160]:
print(df['Cep'])

0      65062630
1      95670000
2      08022120
3      08390355
4      97015070
         ...   
184    65248000
185    60170241
186    72220060
187    82700350
189    72220060
Name: Cep, Length: 179, dtype: object


#### Retornando Estado, Cidade e Bairro referente ao CEP

In [161]:
import requests
import time

sigla_para_estado = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas',
    'BA': 'Bahia', 'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo',
    'GO': 'Goiás', 'MA': 'Maranhão', 'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba', 'PR': 'Paraná',
    'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}

df['Cidade'] = ''
df['Estado'] = ''
df['Bairro'] = ''
df['É São Paulo?'] = ''
df['Zona'] = ''

for i, cep in enumerate(df['Cep']):
    try:
        # response = requests.get(f"https://viacep.com.br/ws/{cep}/json/") viacep
        response = requests.get(f"https://cep.awesomeapi.com.br/json/{cep}") # AwesomeAPI

        data = response.json()

        if 'erro' not in data:
            df.at[i, 'Cidade'] = data.get('city', '')
            sigla_uf = data.get('state', '')
            df.at[i, 'Estado'] = sigla_para_estado.get(sigla_uf, sigla_uf)

            bairro = data.get('district')
            df.at[i, 'Bairro'] = bairro if bairro else 'Não informado'
        else:
            print(f"CEP inválido: {cep}")
        
        time.sleep(0.2)

    except Exception as e:
        print(f"Erro no CEP {cep}: {e}")


#### Movendo Colunas relevantes para novo excel

In [162]:
colunas_desejadas = ['Cep', 'Estado', 'Cidade', 'Bairro', 'É São Paulo?', 'Zona', 'Número de sacos de lixo', 'Litragem dos sacos', 'Peso total dos resíduos', 'Quantidade total de itens coletados']
df = df[colunas_desejadas]

#### Eliminando CEPs não existentes

In [163]:
ceps_remover = ['95670000', '36800000', '12120000', '41603270', '60000000', '74030000']
df = df[~df['Cep'].isin(ceps_remover)]

#### Gerando Coluna de True ou False para São Paulo

In [164]:
df['É São Paulo?'] = df['Cidade'].str.strip().str.lower().apply(lambda x: 'Sim' if x == 'são paulo' else 'Não')

#### Arrumando valores nulos e padronizando tipos de dados

In [165]:
import pandas as pd
import numpy as np
import re

def extrair_inteiro(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if 'não informado' in valor or not re.search(r'\d+', valor):
        return np.nan
    numeros = re.findall(r'\d+', valor)
    return int(numeros[0]) if numeros else np.nan

def extrair_float(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if 'não informado' in valor or not re.search(r'\d', valor):
        return np.nan
    return np.nan

df['Litragem dos sacos'] = df['Litragem dos sacos'].apply(extrair_inteiro)
df['Número de sacos de lixo'] = df['Número de sacos de lixo'].apply(extrair_inteiro)
df['Quantidade total de itens coletados'] = df['Quantidade total de itens coletados'].apply(extrair_inteiro)
df['Peso total dos resíduos'] = df['Peso total dos resíduos'].apply(extrair_float)


#### Gerando Coluna para Zona de São Paulo

In [166]:
zonas = {
    "Centro": [
        "Sé", "República", "Bom Retiro", "Barra Funda",
        "Santa Cecília", "Consolação", "Bela Vista",
        "Liberdade", "Cambuci"
    ],
    "Zona Norte": [
        "Santana", "Vila Maria", "Tucuruvi", "Jaçanã", "Tremembé",
        "Mandaqui", "Casa Verde", "Cachoeirinha", "Limão",
        "Brasilândia", "Freguesia do Ó", "Pirituba",
        "São Domingos", "Jaraguá", "Perus", "Horto Florestal"
    ],
    "Zona Leste": [
        "Belém", "Brás", "Pari", "Mooca", "Vila Prudente", "Água Rasa",
        "São Lucas", "Sapopemba", "Tatuapé", "Vila Formosa", "Vila Carrão",
        "Vila Matilde", "Artur Alvim", "Penha", "Cangaiba", "Ponte Rasa",
        "Ermelino Matarazzo", "São Mateus", "São Miguel",
        "Itaim Paulista", "Itaquera", "Guaianazes", "Jardim Santo André", "Cidade Patriarca", "Vila Gomes Cardim"
    ],
    "Zona Sul": [
        "Vila Mariana", "Moema", "Saúde", "Ipiranga (Vila Monumento)",
        "Vila Mariana (Jardim da Glória)", "Ipiranga", "Sacomã", "Jabaquara",
        "Cidade Ademar", "Pedreira", "Itaim Bibi", "Campo Belo",
        "Santo Amaro", "Cidade Dutra", "Grajaú", "Parelheiros",
        "Capela do Socorro", "Campo Limpo", "Capão Redondo","Jardim Celeste", "Vila Monte Alegre", "Vila Clara", "Parque do Lago", "Jardim Catanduva", "Jardim Figueira Grande", "Chácara da Enseada", "Cidade Ipava", "Vila Santa Lúcia", "Morro do Índio", "Mirandópolis"
    ],
    "Zona Oeste": [
        "Jardim Paulista", "Lapa", "Perdizes", "Jaguara",
        "Vila Leopoldina", "Jaguaré", "Rio Pequeno",
        "Pinheiros", "Alto de Pinheiros", "Butantã", "Morumbi", "Vila Sônia", "Cidade Monções"
    ]
}

def buscar_zona(row):
    cidade = str(row['Cidade']).strip().lower()
    bairro = str(row['Bairro']).strip().lower()

    if cidade != 'são paulo' or not bairro:
        return ''

    for nome_zona, bairros in zonas.items():
        for b in bairros:
            if b.lower() in bairro:
                return nome_zona
    return 'Desconhecida'

df['Zona'] = df.apply(buscar_zona, axis=1)


#### Gerando Excel

In [167]:
df.to_excel('limpaBrasilOrganizado.xlsx', index=False) 

In [168]:
print(df)

          Cep             Estado       Cidade                   Bairro  \
0    65062630           Maranhão     São Luís                  Angelim   
2    08022120          São Paulo    São Paulo               Vila Clara   
3    08390355          São Paulo    São Paulo       Jardim Santo André   
4    97015070  Rio Grande do Sul  Santa Maria  Nossa Senhora de Fátima   
5    04052020          São Paulo    São Paulo             Mirandópolis   
..        ...                ...          ...                      ...   
156       NaN              Ceará    Fortaleza          Dionisio Torres   
165       NaN         Pernambuco  Rio Formoso            Não informado   
166       NaN          São Paulo    São Paulo                Liberdade   
175       NaN              Ceará    Fortaleza          Dionisio Torres   
178       NaN   Distrito Federal     Brasília           Ceilândia Sul    

    É São Paulo?        Zona  Número de sacos de lixo  Litragem dos sacos  \
0            Não                  

In [169]:
# Não foi rodado ainda!